In [1]:
import requests
import pandas as pd
import duckdb
import json
import datamart_profiler
id="datamart.zenodo.3749559.e7fe094d-2f11-4814-ac1a-3326902e063c"
filename=id.replace(".","_").replace("-","_")+".parquet"
resp=requests.get('https://auctus.vida-nyu.org/api/v1/download/'+id)

In [2]:
data=resp.content.decode()
list=[x.split(',') for x in data.split('\n')]

df=pd.DataFrame(list[1:],columns=list[0])


In [3]:
profiles=datamart_profiler.process_dataset(df)
for c in profiles['columns']:
    dtypes=[]
    dtypes.append(c['structural_type'].split('/')[-1])
    for s in c['semantic_types']:
        dtypes.append(s.split('/')[-1])
    print(c['name'],dtypes)

/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value 

country ['Text', 'Enumeration']
date ['Text', 'Enumeration', 'DateTime']
total_cases ['Integer']
new_cases ['Integer']
total_deaths ['Integer']
new_deaths ['Integer']
total_recovered ['Integer']
active_cases ['Integer']
servious_critical ['Integer']
total_cases_1M_pop ['Float']
total_deaths_1M_pop ['Float']
total_tests ['Integer']
 ['Text']pop


/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [4]:
df.to_parquet(filename)
df.to_csv('test_file.csv')  

In [5]:
con=duckdb.connect("testdb.db")
con.sql(" describe select * from "+filename).fetchdf()

,column_name,column_type,null,key,default,extra
0,country,VARCHAR,YES,NaN,NaN,NaN
1,date,VARCHAR,YES,NaN,NaN,NaN
2,total_cases,VARCHAR,YES,NaN,NaN,NaN
3,new_cases,VARCHAR,YES,NaN,NaN,NaN
4,total_deaths,VARCHAR,YES,NaN,NaN,NaN
5,new_deaths,VARCHAR,YES,NaN,NaN,NaN
6,total_recovered,VARCHAR,YES,NaN,NaN,NaN
7,active_cases,VARCHAR,YES,NaN,NaN,NaN
8,servious_critical,VARCHAR,YES,NaN,NaN,NaN
9,total_cases_1M_pop,VARCHAR,YES,NaN,NaN,NaN


In [6]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [11]:
df_cpy=df.copy()
df_cpy
profiles=datamart_profiler.process_dataset(df_cpy)
for c in profiles['columns']:
    dtypes=[]
    dtypes.append(c['structural_type'].split('/')[-1])
    for s in c['semantic_types']:
        dtypes.append(s.split('/')[-1])
    try:
        if "DateTime" in dtypes:
            df_cpy[c['name']] = pd.to_datetime(df_cpy[c['name']], format='%Y-%m-%d')
        elif "Integer" in dtypes:
            df_cpy[c['name']]=df_cpy[c['name']].fillna(0).astype(float).astype(int)
        elif "Float" in dtypes:
            df_cpy[c['name']]=df_cpy[c['name']].fillna(0).astype(float)
        elif "Text" in dtypes:
            df_cpy[c['name']]=df_cpy[c['name']].fillna("null").astype(str)
    except:
        pass


/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/aromston/software/anaconda3/envs/spark/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value 

In [12]:
df_cpy.dtypes

country                        object
date                   datetime64[ns]
total_cases                     int64
new_cases                       int64
total_deaths                    int64
new_deaths                      int64
total_recovered                 int64
active_cases                    int64
servious_critical               int64
total_cases_1M_pop            float64
total_deaths_1M_pop           float64
total_tests                     int64
tests_1M_pop\r                 object
dtype: object

In [13]:
df_cpy.to_parquet("dummy_p.parquet")

In [14]:
con.sql(" describe select * from dummy_p.parquet ")

┌─────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name     │ column_type │  null   │   key   │ default │  extra  │
│       varchar       │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ country             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ date                │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ total_cases         │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ new_cases           │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ total_deaths        │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ new_deaths          │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ total_recovered     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ active_cases        │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ servious_critical   │ BIGINT      │ YES     │ NULL    │ NULL  

In [18]:
con.sql(" select date,count(*) from dummy_p.parquet group by date")

┌─────────────────────┬──────────────┐
│        date         │ count_star() │
│      timestamp      │    int64     │
├─────────────────────┼──────────────┤
│ 2020-03-24 00:00:00 │          195 │
│ 2020-03-25 00:00:00 │          199 │
│ 2020-03-26 00:00:00 │          200 │
│ 2020-03-27 00:00:00 │          201 │
│ 2020-03-28 00:00:00 │          201 │
│ 2020-03-29 00:00:00 │          201 │
│ 2020-03-30 00:00:00 │          202 │
│ 2020-03-31 00:00:00 │          204 │
│ 2020-04-01 00:00:00 │          205 │
│ 2020-04-02 00:00:00 │          206 │
│ 2020-04-03 00:00:00 │          207 │
│ 2020-04-04 00:00:00 │          208 │
│ 2020-04-05 00:00:00 │          210 │
│ 2020-04-06 00:00:00 │          211 │
│ 2020-04-07 00:00:00 │          211 │
│ 2020-04-08 00:00:00 │          209 │
│ 2020-04-09 00:00:00 │          209 │
│ 2020-04-10 00:00:00 │          210 │
│ 2020-04-11 00:00:00 │          210 │
│ 2020-04-12 00:00:00 │          210 │
│ NULL                │            1 │
├─────────────────────┴──

In [24]:
con.sql("select date,count(*) from dummy_p.parquet group by date having MONTH(date) between 1 and 2 ")

┌───────────┬──────────────┐
│   date    │ count_star() │
│ timestamp │    int64     │
├──────────────────────────┤
│          0 rows          │
└──────────────────────────┘

In [35]:
con.sql("select date from datamart_zenodo_3749559_e7fe094d_2f11_4814_ac1a_3326902e063c.parquet").to_df().to_json(orient='records')

'[{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1585008000000},{"date":1